In [1]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
import numpy as np
import time
import qgrid

In [2]:
def loadCSV(pathSamples, pathMatrix):
    df_loaded = pd.read_table(pathMatrix, sep=',')
    data = pd.read_table(pathSamples, sep=',')
    clin_trial_values = df_loaded.values
    columns = df_loaded.columns
    
    Y = data['Eligible']
    Y = Y.astype(int)
    X = clin_trial_values[:, :]
    return X,Y, columns

In [3]:
X, Y, columns = loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1200.csv") #Cargar SCV

In [4]:
print(Y)

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    1
9998    1
9999    1
Name: Eligible, Length: 10000, dtype: int32


In [5]:
from sklearn.model_selection import train_test_split 
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2) # Modificar metodología de validación

In [6]:
import xgboost
from sklearn.metrics import accuracy_score

def xgBoost(X,Y,learning_rate=0.1,  colsample_bytree = 0.3, max_depth = 5, alpha=10, n_estimators = 20, objective = 'reg:squarederror'):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2)
    time_finish = time.time()
    # cargamos las 4 combinaciones de las compuertas XOR
    training_data = Xtrain

    # y estos son los resultados que se obtienen, en el mismo orden
    target_data = Ytrain
    
    model = xgboost.XGBClassifier(objective ='binary:logistic', colsample_bytree = colsample_bytree, learning_rate = learning_rate,
                    max_depth = max_depth, alpha = alpha, n_estimators = n_estimators, booster = 'gbtree')


    model.fit(training_data, target_data, eval_metric='error')
    
    preds = model.predict(Xtest)
    pred_train = model.predict(training_data)
    
    accuracyTrain = accuracy_score(target_data.values, pred_train)
    
    accuracyTest = accuracy_score(Ytest.values, preds)

    return model, str(time.time() - time_finish), accuracyTrain, accuracyTest

In [9]:
pd.options.mode.chained_assignment = None
import time
import qgrid

randn = np.random.randn
df_types = pd.DataFrame({
    'learning_rate' : pd.Series([0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4]), 
    'colsample_bytree' : pd.Series([0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3]), 
    'max_depth' : pd.Series([1,2,1,2,1,2,1,2,1,2]),
    'alpha' : pd.Series([10,10,10,10,10,10,10,10,10,10]),
    'n_estimators' : pd.Series([10,20,10,20,10,20,10,20,10,20])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['learning_rate', 'colsample_bytree', 'max_depth','alpha', 'n_estimators'], inplace=True)

for learning_rate, colsample_bytree, max_depth, alpha, n_estimators in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = xgBoost(X,Y,learning_rate=learning_rate,colsample_bytree = colsample_bytree,
                                                              max_depth = max_depth, alpha=alpha, n_estimators = n_estimators)   
    
    
    df_types["Accuracy test"][learning_rate, colsample_bytree, max_depth, alpha] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][learning_rate, colsample_bytree, max_depth, alpha] = time_finish
    df_types["Accuracy train"][learning_rate, colsample_bytree, max_depth, alpha] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

C:\Users\danie\Anaconda2\envs\newenv\lib\site-packages\IPython\core\interactiveshell.py:2886: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


Accuracy train  \
learning_rate colsample_bytree max_depth alpha n_estimators                  
0.4           0.3              1         10    10                  65.0625   
                               2         10    20                   72.975   
                               1         10    10                  65.0625   
                               2         10    20                   72.975   
                               1         10    10                  65.0625   
                               2         10    20                   72.975   
                               1         10    10                  65.0625   
                               2         10    20                   72.975   
                               1         10    10                  65.0625   
                               2         10    20                   72.975   

                                                            Accuracy test  \
learning_rate colsample_bytree max_depth alpha n_estimators                 
0.4           0.3              1         10    10                   61.65   
                               2         10    20                    68.1   
                               1         10    10                   61.65   
                               2         10    20                    68.1   
                               1         10    10                   61.65   
                               2         10    20                    68.1   
                               1         10    10                   61.65   
                               2         10    20                    68.1   
                               1         10    10                   61.65   
                               2         10    20                    68.1   

                                                            Tiempo de ejecución  
learning_rate colsample_bytree max_depth alpha n_estimators                      
0.4           0.3              1         10    10              1.01601243019104  
                               2         10    20            1.6663193702697754  
                               1         10    10              1.01601243019104  
                               2         10    20            1.6663193702697754  
                               1         10    10              1.01601243019104  
                               2         10    20            1.6663193702697754  
                               1         10    10              1.01601243019104  
                               2         10    20            1.6663193702697754  
                               1         10    10              1.01601243019104  
                               2         10    20            1.6663193702697754